In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../../python-tools'))
	print(os.getcwd())
except:
	pass


 ### 1. Changing string representation
 * To change the string representation of an instance, define the **__str__()** and ** __repr__()** methods.

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    # The format code {0.x} specifies the x-attribute of argument 0.
    # So, in the following function, the 0 is actually the instance self
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)

    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)


In [2]:
p = Pair(3,4)
p   # __repr__() output


Pair(3, 4)

In [3]:
print(p)    # __str__() output


(3, 4)


In [4]:
# the special !r formatting code indicates that the output of __repr__()
# should be used instead of __str__(), the default
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))


p is Pair(3, 4)
p is (3, 4)


In [5]:
# alternative way to this implementation, use the % operator as follows:
def __repr__(self):
    return 'Pair(%r, %r)' % (self.x, self.y)


 ### 2. Customizing string formatting
 * define the __format__() method on a class

In [6]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
}


In [7]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)



In [8]:
d = Date(2012, 12, 21)
format(d)


'2012-12-21'

In [9]:
format(d, 'mdy')


'12/21/2012'

In [10]:
'The date is {:ymd}'.format(d)


'The date is 2012-12-21'

In [11]:
'The date is {:mdy}'.format(d)


'The date is 12/21/2012'

In [12]:
from datetime import date


In [13]:
d = date(2019, 10, 1)
format(d)


'2019-10-01'

In [14]:
format(d,'%A, %B %d, %Y')


'Tuesday, October 01, 2019'

In [15]:
'The end is {:%d %b %Y}.'.format(d)


'The end is 01 Oct 2019.'

 ### 3. Saving memory when creating a large number of instances
 * adding the __slots__ attribute to the class definition

In [16]:
# Attribute names listed in the __slots__ specifier are internally mapped to
# specific indices within this array.
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
            self.year = year
            self.month = month
            self.day = day


 ### 4. Customizing access to an attribute

In [17]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    # getter function
    @property
    def first_name(self):
        return self._first_name

    # setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # delete function
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")


In [18]:
a = Person('Mike')


In [19]:
a.first_name    # Calls the getter


'Mike'

In [20]:
a.first_name = 42   # Calls the setter


TypeError: Expected a string